# Load database

In [6]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv

In [7]:
# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter caminhos do arquivo csv e do diretório de extração das variáveis de ambiente
contas_path = os.getenv('CONTAS_CSV')

In [8]:
# Carregar o CSV em um DataFrame Pandas
df_contas = pd.read_csv(contas_path)

**Wrangling**

Vamos realizar a manipulação dos dados para o dashboard, priorizando a obtenção de uma granularidade consistente e aprimoramento da qualidade da base de dados.

In [9]:
df_contas.head()

,num_conta,cod_cliente,cod_agencia,cod_colaborador,tipo_conta,data_abertura,saldo_total,saldo_disponivel,data_ultimo_lancamento
0,53,53,1,7,PF,2011-05-24 15:30:00 UTC,2984.7615,2814.67,2019-07-02 11:03:46 UTC
1,190,190,1,7,PF,2014-10-17 14:48:00 UTC,4262.4322,4191.86,2021-03-25 12:00:15 UTC
2,191,191,1,7,PF,2013-03-23 14:14:00 UTC,3266.5680,3080.71,2016-09-09 23:34:15 UTC
3,203,203,1,7,PF,2016-08-03 16:13:00 UTC,21241.5424,19461.56,2022-12-28 07:58:24 UTC
4,254,254,1,7,PF,2012-10-24 13:48:00 UTC,13335.6966,13197.54,2013-12-20 22:20:00 UTC


In [10]:
df_contas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_conta               999 non-null    int64  
 1   cod_cliente             999 non-null    int64  
 2   cod_agencia             999 non-null    int64  
 3   cod_colaborador         999 non-null    int64  
 4   tipo_conta              999 non-null    object 
 5   data_abertura           999 non-null    object 
 6   saldo_total             999 non-null    float64
 7   saldo_disponivel        999 non-null    float64
 8   data_ultimo_lancamento  999 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 70.4+ KB


Vamos fazer um type casting da coluna data_abertura, data_ultimo_lancamento para o formato de 'datetime64[ns]'

A coluna data_abertura está formatada com o sufixo "UTC". Para lidar com isso, usaremos a função pd.to_datetime() do pandas, que oferece mais flexibilidade para o parsing de strings de data.

In [11]:
# Converter corretamente as strings de data com o sufixo "UTC" para objetos datetime do pandas
df_contas['data_abertura'] = pd.to_datetime(df_contas['data_abertura'], utc=True)

# remover a informação de fuso horário, convertendo as datas para o formato local naive datetime
df_contas['data_abertura'] = df_contas['data_abertura'].dt.tz_localize(None)

In [12]:
df_contas['data_abertura'] = df_contas['data_abertura'].astype('datetime64[ns]')

A coluna data_ultimo_lancamento está formatada com o sufixo "UTC". Para lidar com isso, usaremos a função pd.to_datetime() do pandas, que oferece mais flexibilidade para o parsing de strings de data.

In [14]:
# Converter corretamente as strings de data com o sufixo "UTC" para objetos datetime do pandas
df_contas['data_ultimo_lancamento'] = pd.to_datetime(df_contas['data_ultimo_lancamento'], errors='coerce', utc=True)

# remover a informação de fuso horário, convertendo as datas para o formato local naive datetime
df_contas['data_ultimo_lancamento'] = df_contas['data_ultimo_lancamento'].dt.tz_localize(None)

In [15]:
df_contas['data_ultimo_lancamento'] = df_contas['data_ultimo_lancamento'].astype('datetime64[ns]')

Enriquecimento dos dados com novas colunas.

Chaves temporais:

In [16]:
df_contas['data_abertura_month'] = df_contas['data_abertura'].apply(lambda date: date.strftime('%Y-%m'))
df_contas['data_abertura_year']  = df_contas['data_abertura'].apply(lambda date: date.strftime('%Y'))

Com os dados manipulados, procederemos à sua persistência em disco, realizaremos o download e, posteriormente, o carregaremos no Google Data Studio.

In [18]:
# Obter caminhos do diretório gold das variáveis de ambiente
gold_path = os.getenv('GOLD_PATH')

In [19]:
df_contas.to_csv(f'{gold_path}/contas.csv', sep=',', index=False)